# Fighting COVID-19 Infodemics 
## Project Design

Knowledge gap and uncertainties between general public and specialists are important factors that drive pandemic anxiety. In this Kaggle competition task, we will examine the papers that discuss some of the controversial topics that trigger social rumours and pandenic anxiety in general public during the spread of the pandemic covid-19. 

Ever since the beginning of 2020, there have been are lots of unverified statements about the virus and how to prevent it from spreading circulating in many social media. One way to validate these statements is to examine the relevant academic papers. However, most of the public are struggling to find out the right keywords to search the academic papers, and they are usually reluctant to go through the entire papers to search for the relevant information. In this task, we aim to develop a searching system to extract the sentences from abstracts in the CORD-91 dataset that are relevant to evaluate a specific topic.

For each statment, we first use the system to extract the topic relevant sentences from the paper abstracts, and then manually annotate those sentences based on whether they are for or against the statement.

Here are some of the example statements about covid-19:

1. Wearing mask is an effective way to stop the spreading of the virus

2. The incubation period of covid-19 ranges from 2-14 days with a median of 5 days

3. Asymptomatic patients can drive the spread of the virus

For each statement, we define at least two stance levels according to whether the exatracted sentences in the abstracts support or oppose the statement. Here are the steps to achieve this goal:

### Step 1: Developing a topic searching system 

The searching system first extracts abstracts contain a particular keyword (e.g. ‘mask’), then we use LDA to group the abstract topics. We identify a topic that is the most relevant to the statement and extract the abstracts that contain the target topic. The system then extract the sentences that contain the keyword from the relevant abstracts. The standard apporach of a searching system is to use TFIDF to rank documents, here we use LDA topic modeling on nouns, verbs, and adjectives of the abstracts. Users can decide the relevant information when knowing what are the most frequent keywords in each topic. For some queries, users want to identify articles for covid-19 only. Therefore, users are opt to add a title filer for different queries in the system.

The benefit of this approach is that when we want to know the relevant contents for a question, we don't know beforehand what are the keywords in an article that are more relevant to the question we ask, because users from general public are usually not farmiliar with academic papers. In our system, the topic keywords serve as the primes for the query in the next step of sentence extraction in the abstracts.

### Step 2: Mannually annotating stance and relevance

We manually annotate the key sentences to identify the stance of the result sentences and whether these sentences are relevant to the question asked. Relevance annotation is an important part for evaluating a searching system.


#### Annotation results
To understand the answer to the relevant question, we need to annotate the stance of the results, e.g., whether the abstract is for / against the statement. 

To evaluate the searching system, we need to annotate the relevance of the retrieved sentences. Please refer to each section for the annotation guildlines

Retrieved results and annotations can be found in this document 
https://docs.google.com/spreadsheets/d/1-eWEqji7mLXNF0Z9KH8RE5djcxK-97dUHzPWY7GEhI8/edit?usp=sharing

The document contains:

1. Annotation of stance: See the column 'stance' in the sheets 'mask', 'incubation', 'asymtomatic', 

2. Annotation for relevance: See the column 'relevance' in the sheets 'mask', 'incubation', 'asymtomatic', 

3. Annotation for system evaluation: See the column 'relevance' in the sheets 'system_eval_varname'. 


### Results:

#### Statement 1: Wearing mask is an effective way to stop the spreading of the virus

According to the key sentences in the 40 paper abstracts that discuss the topic of public using masks, 12 papers support that using a mask during a pandemic is useful, 18 papers assume masks to be useful and examine the public’s willingness to comply the mask wearing rules, 1 paper shows no obvious evidence that using mask is protective or the protection is very limited.

We also see that the governments in some regions advocate using masks as a standard approach to reduce the risk of infection, papers in these regions focus on whether people comply the rules. Some government advocate that there is little evidence showing that mask is effective in controlling the pandemic, whereas nearly half of the academic papers from our search results consider wearing masks as a standard practice and these papers examine whether the public comply the recommended practice. Another half of the papers found evidence to support that wearing masks is effective in controlling the pandemic.


#### Statement 2: The incubation period range from 2-14 days with a media of 5 days

There are 16 papers showing that the incubation period of covid-19 is 2-14 days with median of 5 days, 51 papers show different numbers. We can see that the majority of the papers show slightly different results than what the authorities reported. 

#### Statement 3: Asymptomatic patients drive the spread of the virus

According to 53 papers relevant to this topic, 28 papers show that there is clear evidence that asymtomatic cases contribute to the spread of the virus, 25 papers show that it is unlikely that asymtomatic cases contribute to the spread of the virus. Therefore, more research is needed for this topic. As general public, one need to take some precautions when contacting others who might be asymptomatic patients.


### Evaluation of the system:

The evaluation of the system is currently being implemented. A test collection will be annotated in order to compare our search system with a baseline system.


In [ ]:
import pandas as pd 
from collections import defaultdict
import string
from gensim.models import CoherenceModel
import gensim
from pprint import pprint
import spacy,en_core_web_sm
from nltk.stem import PorterStemmer
import os
import json
from gensim.models import Word2Vec
import nltk
import re
import collections
from sklearn.metrics import cohen_kappa_score
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from collections import defaultdict
import string

In [ ]:
### Read the metadata in a dictionary format

In [ ]:
class MetaData:
    def __init__(self):
        """Define varibles."""
        # path and data
        self.path = '../input/CORD-19-research-challenge/'
        self.meta_data = pd.read_csv(self.path + 'metadata.csv')

    def data_dict(self):
        """Convert df to dictionary. """
        mydict = lambda: defaultdict(mydict)
        meta_data_dict = mydict()

        for cord_uid, abstract, title, sha in zip(self.meta_data['cord_uid'], self.meta_data['abstract'], self.meta_data['title'], self.meta_data['sha']):
            meta_data_dict[cord_uid]['title'] = title
            meta_data_dict[cord_uid]['abstract'] = abstract
            meta_data_dict[cord_uid]['sha'] = sha

        return meta_data_dict

### Extract the documents containing keywords, preprocessing 

In [ ]:
class ExtractText:
    """Extract text according to keywords or phrases"""

    def __init__(self, metaDict, keyword, variable):
        """Define varibles."""
        self.path = '../input/CORD-19-research-challenge/'
        self.metadata = metaDict
        self.keyword = keyword
        self.variable = variable


    def simple_preprocess(self):
        """Simple text process: lower case, remove punc. """
        mydict = lambda: defaultdict(mydict)
        cleaned = mydict()
        for k, v in self.metadata.items():
            sent = v[self.variable]
            sent = str(sent).lower().translate(str.maketrans('', '', string.punctuation))
            cleaned[k]['processed_text'] = sent
            cleaned[k]['sha'] = v['sha']
            cleaned[k]['title'] = v['title']

        return cleaned

    def very_simple_preprocess(self):
        """Simple text process: lower case only. """
        mydict = lambda: defaultdict(mydict)
        cleaned = mydict()
        for k, v in self.metadata.items():
            sent = v[self.variable]
            sent = str(sent)
            #sent = str(sent).lower()
            cleaned[k]['processed_text'] = sent
            cleaned[k]['sha'] = v['sha']
            cleaned[k]['title'] = v['title']

        return cleaned
     

    def extract_w_keywords(self):
        """Select content with keywords."""
        ps = PorterStemmer()
        mydict = lambda: defaultdict(mydict)
        selected = mydict()
        textdict = self.simple_preprocess()
        
        for k, v in textdict.items():
            if self.keyword in v['processed_text'].split():
                #print(ps.stem(str(self.keyword)))
                selected[k]['processed_text'] = v['processed_text']
                selected[k]['sha'] = v['sha']
                selected[k]['title'] = v['title']
        return selected

    def extract_w_keywords_punc(self):
        """Select content with keywords, with punctuations in text"""
        ps = PorterStemmer()
        mydict = lambda: defaultdict(mydict)
        selected = mydict()
        textdict = self.very_simple_preprocess()
        
        for k, v in textdict.items():
            #keywords are stemmed before matching
            if ps.stem(str(self.keyword)) in ps.stem(str(v['processed_text'].split())):
                selected[k]['processed_text'] = v['processed_text']
                selected[k]['sha'] = v['sha']
                selected[k]['title'] = v['title']
        return selected

    def get_noun_verb(self, text):
        """get noun trunks for the lda model,
        change noun and verb part to decide what
        you want to use as input for LDA"""
        ps = PorterStemmer()
      
        #find nound trunks
        nlp = en_core_web_sm.load()
        all_extracted = {}
        for k, v in text.items():
            #v = v.replace('incubation period', 'incubation_period')
            doc = nlp(v)
            nouns = ' '.join(str(v) for v in doc if v.pos_ is 'NOUN').split()
            verbs = ' '.join(ps.stem(str(v)) for v in doc if v.pos_ is 'VERB').split()
            adj = ' '.join(str(v) for v in doc if v.pos_ is 'ADJ').split()
            all_w = nouns + verbs + adj
            all_extracted[k] = all_w
      
        return all_extracted

    def get_noun_verb2(self, text):
        """get noun trunks for the lda model,
        change noun and verb part to decide what
        you want to use as input for LDA"""
        ps = PorterStemmer()
      
        #find nound trunks
        nlp = en_core_web_sm.load()
        all_extracted = {}
        for k, v in text.items():
            #v = v.replace('incubation period', 'incubation_period')
            doc = nlp(v['processed_text'])
            nouns = ' '.join(ps.stem(str(v)) for v in doc if v.pos_ is 'NOUN').split()
            verbs = ' '.join(ps.stem(str(v)) for v in doc if v.pos_ is 'VERB').split()
            adj = ' '.join(str(v) for v in doc if v.pos_ is 'ADJ').split()
            all_w = nouns + verbs + adj
            all_extracted[k] = all_w
      
        return all_extracted

    def tokenization(self, text):
        """get noun trunks for the lda model,
        change noun and verb part to decide what
        you want to use as input for the next step"""
        nlp = spacy.load("en_core_web_sm")

        all_extracted = {}
        for k, v in text.items():
            doc = nlp(v)
            all_extracted[k] = [w.text for w in doc]
      
        return all_extracted



## Using LDA to rank documents
LDA is optimized by coherence score u_mass

In [ ]:
class LDATopic:
    def __init__(self, processed_text, topic_num, alpha, eta):
        """Define varibles."""
        self.path = '../input/CORD-19-research-challenge/'
        self.text = processed_text
        self.topic_num = topic_num
        self.alpha = alpha
        self.eta = eta

    def get_lda_score_eval(self, dictionary, bow_corpus):
        """LDA model and coherence score."""

        lda_model = gensim.models.ldamodel.LdaModel(bow_corpus, num_topics=self.topic_num, id2word=dictionary, passes=10,  update_every=1, random_state = 300, alpha=self.alpha, eta=self.eta)
        #pprint(lda_model.print_topics())

        # get coherence score
        cm = CoherenceModel(model=lda_model, corpus=bow_corpus, coherence='u_mass')
        coherence = cm.get_coherence()
        print('coherence score is {}'.format(coherence))

        return lda_model, coherence

    def get_score_dict(self, bow_corpus, lda_model_object):
        """
        get lda score for each document
        """
        all_lda_score = {}
        for i in range(len(bow_corpus)):
            lda_score ={}
            for index, score in sorted(lda_model_object[bow_corpus[i]], key=lambda tup: -1*tup[1]):
                lda_score[index] = score
                od = collections.OrderedDict(sorted(lda_score.items()))
            all_lda_score[i] = od
        return all_lda_score


    def topic_modeling(self):
        """Get LDA topic modeling."""
        # generate dictionary
        dictionary = gensim.corpora.Dictionary(self.text.values())
        bow_corpus = [dictionary.doc2bow(doc) for doc in self.text.values()]
        # modeling
        model, coherence = self.get_lda_score_eval(dictionary, bow_corpus)

        lda_score_all = self.get_score_dict(bow_corpus, model)

        all_lda_score_df = pd.DataFrame.from_dict(lda_score_all)
        all_lda_score_dfT = all_lda_score_df.T
        all_lda_score_dfT = all_lda_score_dfT.fillna(0)

        return model, coherence, all_lda_score_dfT

    def get_ids_from_selected(self, text):
        """Get unique id from text """
        id_l = []
        for k, v in text.items():
            id_l.append(k)
            
        return id_l

In [ ]:
# Now we extract articles contain the most relevant topic

def selected_best_LDA(keyword, varname):
        """Select the best lda model with extracted text """
        # convert data to dictionary format
        m = MetaData()
        metaDict = m.data_dict()

        #process text and extract text with keywords
        et = ExtractText(metaDict, keyword, varname)
        text1 = et.extract_w_keywords()


        # extract nouns, verbs and adjetives
        text = et.get_noun_verb2(text1)

        # optimized alpha and beta
        alpha = [0.1, 0.3, 0.5, 0.7, 0.9]
        beta = [0.1, 0.3, 0.5, 0.7, 0.9]

        mydict = lambda: defaultdict(mydict)
        cohere_dict = mydict()
        for a in alpha:
            for b in beta:
                lda = LDATopic(text, 20, a, b)
                model, coherence, scores = lda.topic_modeling()
                cohere_dict[coherence]['a'] = a
                cohere_dict[coherence]['b'] = b
    
        # sort result dictionary to identify the best a, b
        # select a,b with the largest coherence score 
        sort = sorted(cohere_dict.keys())[0] 
        a = cohere_dict[sort]['a']
        b = cohere_dict[sort]['b']
        
        # run LDA with the optimized values
        lda = LDATopic(text, 20, a, b)
        model, coherence, scores_best = lda.topic_modeling()
        pprint(model.print_topics())

        # select merge ids with the LDA topic scores
        id_l = lda.get_ids_from_selected(text)
        scores_best['cord_uid'] = id_l

        return scores_best




def select_text_from_LDA_results(keyword, varname, scores_best, topic_num):
        # choose papers with the most relevant topic
        # convert data to dictionary format
        m = MetaData()
        metaDict = m.data_dict()

        # process text and extract text with keywords
        et = ExtractText(metaDict, keyword, varname)
        # extract text together with punctuation
        text1 = et.extract_w_keywords_punc()
        # need to decide which topic to choose after training
        sel = scores_best[scores_best[topic_num] > 0] 
        
        mydict = lambda: defaultdict(mydict)
        selected = mydict()
        for k, v in text1.items():
            if k in sel.cord_uid.tolist():
                selected[k]['title'] = v['title']
                selected[k]['processed_text'] = v['processed_text']
                selected[k]['sha'] = v['sha']
    
        return selected

def extract_relevant_sentences(cor_dict, search_keywords, filter_title=None):
    """Extract sentences contain keyword in relevant articles. """
    #here user can also choose whether they would like to only select title contain covid keywords

    mydict = lambda: defaultdict(mydict)
    sel_sentence = mydict()
    filter_w = ['covid19','ncov','2019-ncov','covid-19','sars-cov','wuhan']
    
    for k, v in cor_dict.items():
        keyword_sentence = []
        sentences = v['processed_text'].split('.')
        for sentence in sentences:
            # for each sentence, check if keyword exist
            # append sentences contain keyword to list
            keyword_sum = sum(1 for word in search_keywords if word in sentence)
            if keyword_sum > 0:
                keyword_sentence.append(sentence)         

        # store results
        if not keyword_sentence:
            pass
        elif filter_title is not None:
            for f in filter_w:
                title = v['title'].lower().translate(str.maketrans('', '', string.punctuation))
                abstract = v['processed_text'].lower().translate(str.maketrans('', '', string.punctuation))
                if (f in title) or (f in abstract):
                    sel_sentence[k]['sentences'] = keyword_sentence
                    sel_sentence[k]['sha'] = v['sha']
                    sel_sentence[k]['title'] = v['title'] 
        else:
            sel_sentence[k]['sentences'] = keyword_sentence
            sel_sentence[k]['sha'] = v['sha']
            sel_sentence[k]['title'] = v['title'] 
            
    print('{} articles are relevant to the topic you choose'.format(len(sel_sentence)))

    path = '../input/CORD-19-research-challenge/'
    df = pd.DataFrame.from_dict(sel_sentence, orient='index')
    #df.to_csv(path + 'search_results_{}.csv'.format(search_keywords))
    #sel_sentence_df = pd.read_csv(path + 'search_results_{}.csv'.format(search_keywords))
    return sel_sentence, df

def extract_relevant_sentences2(cor_dict, search_keywords, filter_title=None):
    """Extract sentences contain keyword in relevant articles for system evaluation. """
    #here user can also choose whether they would like to only select title contain covid keywords
    #difference from the previous one is where we store the result

    mydict = lambda: defaultdict(mydict)
    sel_sentence = mydict()
    filter_w = ['covid19','ncov','2019-ncov','covid-19','sars-cov','wuhan']
    
    for k, v in cor_dict.items():
        keyword_sentence = []
        sentences = v['processed_text'].split('.')
        for sentence in sentences:
            # for each sentence, check if keyword exist
            # append sentences contain keyword to list
            keyword_sum = sum(1 for word in search_keywords if word in sentence)
            if keyword_sum > 0:
                keyword_sentence.append(sentence)         

        # store results
        if not keyword_sentence:
            pass
        
        elif filter_title is not None:
            for f in filter_w:
                title = v['title'].lower().translate(str.maketrans('', '', string.punctuation))
                abstract = v['processed_text'].lower().translate(str.maketrans('', '', string.punctuation))
                if (f in title) or (f in abstract):
                    sel_sentence[k]['sentences'] = keyword_sentence
                    sel_sentence[k]['sha'] = v['sha']
                    sel_sentence[k]['title'] = v['title'] 
        else:
            sel_sentence[k]['sentences'] = keyword_sentence
            sel_sentence[k]['sha'] = v['sha']
            sel_sentence[k]['title'] = v['title'] 
    print('{} articles contain keyword {}'.format(len(sel_sentence),  search_keywords))

    path = '../input/CORD-19-research-challenge/'
    df = pd.DataFrame.from_dict(sel_sentence, orient='index')
    df.to_csv(path + 'eval_results_{}.csv'.format(search_keywords))
    sel_sentence_df = pd.read_csv(path + 'eval_results_{}.csv'.format(search_keywords))
    return sel_sentence, sel_sentence_df


## Question 1: Is wearing mask an effective way to control the pandemic?

In [ ]:
#here we select the LDA model with the lowe
scores_best_mask = selected_best_LDA('mask', 'abstract')

In [ ]:
scores_best_mask.shape

In [ ]:
# topic number 10 is most relevant to public wearing mask
# which topic do you think is most relevant to your search
cor_dict_mask = select_text_from_LDA_results('mask', 'abstract', scores_best_mask, 10)
print ("There are {} abstracts selected". format(len(cor_dict_mask)))

In [ ]:
# extract relevant sentences  #search keywords can be a list
sel_sentence_mask, sel_sentence_df_mask = extract_relevant_sentences(cor_dict_mask, ['mask'])

In [ ]:
#read extracted article
sel_sentence_df_mask.head(20)

### Annotation guidline for question 1
We extracted 33 papers that are supposed to discuss whether using masks is useful. We annotate  whether the key sentences suggest using mask can reduce the risk of infection.

#### Stance Annotation 
* '1' support using a mask during a pandemic is useful 
* '2' assume masks as useful and examine the public’s willingness to comply the rules,
* '0' no obvious evidence that shows using mask is protective or the protection is very little
* '3' not relevant to the above stance

result from annotator 1

#### Relevance annotation
* '1' the result is relevent to the question  
* '0' the result is not relevant to the question

In [ ]:
#here we annotated a sample of 40 abstracts
path = '../input/annotation/'
annotation_mask = pd.read_csv(path + 'wear_mask.csv')

In [ ]:
# view file
annotation_mask.head(5)
print('there are {} articles relevant to the topic'.format(annotation_mask.shape[0]))

In [ ]:
annotation_mask['stance'].value_counts()

In [ ]:
print('there are {} papers support using a mask during a pandemic is useful, {} assume masks as useful and examine the public’s willingness to comply the rules,  {} papers show no obvious evidence that shows using mask is protective or the protection is very little'. format(str(annotation_mask['stance'].value_counts()[1]), str(annotation_mask['stance'].value_counts()[2]), annotation_mask['stance'].value_counts()[0]) )
          

### Question 2: How long in incubation period? In some region (e.g. China), there’s rumour circulating that the incubation period is longer than 14 days

### Annotation guideline for question 2:

#### stance annotation
Here we want to identify papers that report a result aligns with the incubation period reported by the governments
UK government advocated: 2-14 days, median 5
* '1' same as government advocate 
* '0' different from what the government
* Not relevant to the question 

#### relevance annotation
* '1' the result is relevent to the question  
* '0' the result is not relevant to the question


In [ ]:
scores_best_incu = selected_best_LDA('incubation', 'abstract')

In [ ]:
# topic number 0 is most relevant to public wearing mask
# which topic do you think is most relevant to your search
cor_dict_incu = select_text_from_LDA_results('incubation', 'abstract', scores_best_incu, 0)
print ("There are {} abstracts selected". format(len(cor_dict_incu)))

In [ ]:
# extract relevant sentences  #search keywords can be a list
sel_sentence_incu, sel_sentence_df_incu = extract_relevant_sentences(cor_dict_incu, ['incubation','day'], 'title')

In [ ]:
#read extracted article
sel_sentence_df_incu.head(10)

## Statistical analysis on the incubation period

In [ ]:
#here we need to add the stats analysis 
path = '../input/annotation/'
annotation_incubation = pd.read_csv(path + 'incubation.csv')
print('there are {} articles relevant to the topic'.format(annotation_incubation.shape[0]))

In [ ]:
incubation = annotation_incubation['stance'].value_counts()
print('there are {} paper shows the incubation period is 2-14 days with mean 5 days, {} papers shows a different number'. format(incubation[1], incubation[0])
     )

In [ ]:
incubation = annotation_incubation['relevance'].value_counts()
print('there are {} papers relevant to the topic, {} papers not relevant to the topic'. format(incubation[1], incubation[0]))

## Question 3: Are asymptomatic patients infectious?

### Annotation guideline for question 3:
Here we want to identify whether asymtomatic cases contribute to the spread of the virus

#### stance annotation
* ‘1’  there is clear evidence show that asymtomatic cases contribute to the spread of the virus
* ‘0’  it is unlikely that asymtomatic cases contribute to the spread of the virus
* '3' Not relevant to the question

#### relevance annotation
* '1' the result is relevent to the question  
* '0' the result is not relevant to the question

In [ ]:
scores_best_asym = selected_best_LDA('asymptomatic', 'abstract')

In [ ]:
# topic number 8 is most relevant to public wearing mask
# which topic do you think is most relevant to your search
cor_dict_asym = select_text_from_LDA_results('asymptomatic', 'abstract', scores_best_asym, 8)
print ("There are {} abstracts selected". format(len(cor_dict_asym)))

In [ ]:
# extract relevant sentences  #search keywords can be a list
sel_sentence_asym, sel_sentence_df_asym = extract_relevant_sentences(cor_dict_asym, ['asymptomatic','transmission'], 'title')

In [ ]:
sel_sentence_df_asym.tail(10)

## Asymptomatic Result

In [ ]:
#here we need to add the stats analysis 
annotation_asymptomatic = pd.read_csv(path + 'asymtomatic.csv')
print('there are {} articles relevant to the topic'.format(annotation_asymptomatic.shape[0]))

In [ ]:
asymptomatic = annotation_asymptomatic['stance'].value_counts()
print('{} papers show that there is clear evidence show that asymtomatic cases contribute to the spread of the virus, {} papers show that it is unlikely that asymtomatic cases contribute to the spread of the virus'.format(asymptomatic[1], asymptomatic[0]))